Importing The Library

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Collection

In [2]:
movies = pd.read_csv('movies.csv')

In [3]:
movies

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

Top 5 rows of the data

In [4]:
movies.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


Buttom 5 rows of the Data

In [5]:
movies.tail()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,"[{'name': 'Daniel Hsia', 'gender': 2, 'departm...",Daniel Hsia
4802,4802,0,Documentary,NaN,25975,obsession camcorder crush dream girl,en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16,Drew Barrymore Brian Herzlinger Corey Feldman ...,"[{'name': 'Clark Peterson', 'gender': 2, 'depa...",Brian Herzlinger


Finding the rows and column numbers

In [6]:
movies.shape

(4803, 24)

Features Selection

In [7]:
selected_features = ['genres','keywords','tagline','cast','director']

Replacing The Null Values

In [8]:
for feature in selected_features:
    movies[feature] = movies[feature].fillna('')

Combining all the 5 selected features

In [9]:
combined_features = movies['genres'] + movies['keywords'] + movies['tagline'] +movies['cast'] +movies['director'] 

In [10]:
combined_features

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object

Converting the text data to features vector

In [11]:
vectorizer = TfidfVectorizer()

In [12]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [13]:
feature_vectors

<4803x27580 sparse matrix of type '<class 'numpy.float64'>'
	with 111820 stored elements in Compressed Sparse Row format>

Getting the similarity score using cosine similarity

In [14]:
similarity = cosine_similarity(feature_vectors)

In [15]:
similarity

array([[1.        , 0.06865296, 0.01492221, ..., 0.        , 0.        ,
        0.        ],
       [0.06865296, 1.        , 0.02799128, ..., 0.01243107, 0.        ,
        0.        ],
       [0.01492221, 0.02799128, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.01243107, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [16]:
similarity.shape

(4803, 4803)

checking all the movie list

In [17]:
title_list = movies['title'].tolist()

In [18]:
title_list

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

Taking Input From The User

In [19]:
movie_name = input("Enter The Movie Name : ")

Enter The Movie Name : John Carter


Finding The Close Match of the movies

In [20]:
close_movies = difflib.get_close_matches(movie_name,title_list)

In [21]:
close_movies

['John Carter', 'Coach Carter', 'Get Carter']

In [22]:
close_match = close_movies[0]

In [23]:
close_match

'John Carter'

In [24]:
movie_index = movies[movies.title == close_match]['index'].values[0]

In [25]:
movie_index

4

In [26]:
similarity_score = list(enumerate(similarity[movie_index]))

In [27]:
similarity_score

[(0, 0.09507382234152616),
 (1, 0.027865247434327937),
 (2, 0.042741991417021064),
 (3, 0.006632266386061478),
 (4, 1.0),
 (5, 0.07381646175974522),
 (6, 0.008372344059607685),
 (7, 0.03435587404046212),
 (8, 0.008578199220666273),
 (9, 0.024640115964262202),
 (10, 0.04566789427532825),
 (11, 0.013164248585219668),
 (12, 0.008296015257430348),
 (13, 0.013743270173463426),
 (14, 0.050610023914962554),
 (15, 0.03429709540172117),
 (16, 0.024918771275211457),
 (17, 0.013958473138966625),
 (18, 0.06801985922775909),
 (19, 0.012442252630861718),
 (20, 0.013723709220377366),
 (21, 0.005406773323695694),
 (22, 0.007782793094474817),
 (23, 0.037701254822318474),
 (24, 0.04225762077933781),
 (25, 0.010438099654175954),
 (26, 0.025596737211112847),
 (27, 0.07253312842785063),
 (28, 0.02328988882500292),
 (29, 0.02586600983799186),
 (30, 0.036683966592926956),
 (31, 0.03395215233736076),
 (32, 0.04592174544083458),
 (33, 0.034671162067929015),
 (34, 0.0),
 (35, 0.016891073894524655),
 (36, 0.0155

In [28]:
len(similarity_score)

4803

In [29]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

In [30]:
sorted_similar_movies

[(4, 1.0),
 (270, 0.16731006686861688),
 (3494, 0.141252238093673),
 (1473, 0.13541893124958404),
 (3158, 0.1334868495331938),
 (328, 0.1236975690298994),
 (3257, 0.12349286396221273),
 (1748, 0.11640579196141487),
 (1075, 0.11640284979316608),
 (1373, 0.1135841745410553),
 (2904, 0.11146194094053472),
 (2444, 0.11057322534910496),
 (111, 0.11051976193782999),
 (2658, 0.10947883371789935),
 (122, 0.10353282782631618),
 (2246, 0.10158102517451136),
 (2163, 0.10117337499281674),
 (3378, 0.10020698714011919),
 (3875, 0.09737007074197837),
 (1959, 0.0970019055694222),
 (1440, 0.09680141502600932),
 (256, 0.09609809679373998),
 (1158, 0.09576892205572689),
 (305, 0.09569841084670733),
 (4555, 0.0954891002020716),
 (0, 0.09507382234152616),
 (4395, 0.09442684333823184),
 (2769, 0.09424933738720948),
 (400, 0.09219536360463058),
 (369, 0.09213984420376),
 (767, 0.09182599560106365),
 (1326, 0.09124856056918462),
 (275, 0.09081464046672276),
 (1611, 0.09058816889138523),
 (3438, 0.090100680347

Finding Similar movies based on index

In [31]:
print("Movies Suggested for You :\n")

i = 1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies[movies.index==index]['title'].values[0]
    
    if i < 51:
        print(i, '.', title_from_index)
        i = i+1

Movies Suggested for You :

1 . John Carter
2 . The Martian
3 . Beastmaster 2: Through the Portal of Time
4 . The Astronaut's Wife
5 . Alien
6 . Finding Nemo
7 . American Psycho
8 . Notes on a Scandal
9 . Savages
10 . The English Patient
11 . Heaven is for Real
12 . Damnation Alley
13 . Transformers
14 . Pandaemonium
15 . X-Men Origins: Wolverine
16 . Synecdoche, New York
17 . The Covenant
18 . Auto Focus
19 . Dreaming of Joseph Lees
20 . Star Trek IV: The Voyage Home
21 . Flight of the Intruder
22 . Allegiant
23 . Lone Survivor
24 . Treasure Planet
25 . Enter Nowhere
26 . Avatar
27 . The Specials
28 . Paris, je t'aime
29 . Divergent
30 . Lara Croft Tomb Raider: The Cradle of Life
31 . xXx: State of the Union
32 . The 5th Wave
33 . Minority Report
34 . Mortal Kombat: Annihilation
35 . Control
36 . Moonraker
37 . Light Sleeper
38 . Miss Julie
39 . Max
40 . Earth to Echo
41 . eXistenZ
42 . Adam Resurrected
43 . Lost in Space
44 . Sideways
45 . The Merchant of Venice
46 . Star Wars: Clone